In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tripadvisorbanglore/Final_Data1.csv


In [2]:
pip install bert-extractive-summarizer

Note: you may need to restart the kernel to use updated packages.


### Header Files

In [3]:
import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt

# Library to implement BERT
from summarizer import Summarizer

/opt/conda/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


### Importing Data

In [4]:
data=pd.read_csv('../input/tripadvisorbanglore/Final_Data1.csv')
data.drop('Unnamed: 0',axis=1,inplace=True)
data.head()

,Hotel,Review,Date,Ratings
0,Conrad_Bengaluru,My second stay here but still loving every mom...,June 2021,5.0
1,Conrad_Bengaluru,In a neighbourhood filled with a lot of ageing...,June 2021,5.0
2,Conrad_Bengaluru,A really enjoyable staycation. We were celebra...,April 2021,5.0
3,Conrad_Bengaluru,"Absolutely love the staff, they were hospitabl...",April 2021,5.0
4,Conrad_Bengaluru,We visited to Conrad April 2021 to celebrate m...,April 2021,5.0


In [5]:
# Words in one hotel 
reviews=''.join(data[data['Hotel']=='Conrad_Bengaluru']['Review'])
len(reviews)

349758

In [6]:
reviews[:1000]

'My second stay here but still loving every moment of it. Wonderful staff, great hospitality and awesome food. Have been to their Sunday brunch too in January. The spread was oh so extensive. The staff goes an extra mile to make you feel welcome. The rooms are the bestIn a neighbourhood filled with a lot of ageing and also-ran semi-luxury hotels that are way past their glory days, the Conrad is a refreshing option to treat yourself to fine luxury in the heart of the city. I have stayed in Hilton hotels across the globe, including the Conrad brand. While the Conrad in Bangalore comes with all the typical aspects of a Conrad - from Smart luxury to Stylish fittings, this one takes it a few levels higher with Its top-notch service and high quality amenities. And throw in the spectacular views of the charming Ulsoor lake you get from most rooms, you have an indulgent retreat in a striking location - the traffic and construction chatter notwithstanding. Shoutout and kudos to the entire staff

### Implementing Bert

In [7]:
# Initializing the Bert model
model=Summarizer()

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
summary=[]
for x in data.Hotel.unique():
    docs=data[data['Hotel']==x]['Review'].str.lower()
    docs=''.join(docs)
    
    result= model(docs,num_sentences=2,min_length=10)
    summary.append(result)


In [9]:
full_summary=pd.DataFrame()
for i in range(len(summary)):
    new=pd.Series({'Hotel':data.Hotel.unique()[i],'Summary':summary[i]})
    full_summary=full_summary.append(new,ignore_index=True)
    
full_summary

,Hotel,Summary
0,Conrad_Bengaluru,my second stay here but still loving every mom...
1,DoubleTree_Suites_By_Hilton_Bangalore,i had a medical emergency and prakash was very...
2,Gokulam_Grand_Hotel_Spa,stayed for one day...and the rooms were spacio...
3,ITC_Windsor_Bengaluru_a_Luxury_Collection_Hotel,i do not like to write a review usually unless...
4,Keys_Select_Hosur_Road_By_Lemon_Tree_Hotels-Be...,room service was góod and fast. and his team t...
5,Lemon_Tree_Hotel,house keeping is so good..very responsible per...
6,Lemon_Tree_Premier_Ulsoor_Lake_Bengaluru,it was my first visit to bangalore and i chose...
7,Royal_Orchid_Central,"i loved the stay here, i can't wait to come ba..."
8,The_Chancery_Pavilion,"i have been visiting , chancery pavilion, resi..."
9,The_Leela_Palace,the stay was awesome & overall experience was ...


In [10]:
print(full_summary.iloc[2]['Hotel']),full_summary.iloc[2]['Summary']

Gokulam_Grand_Hotel_Spa


(None,
 "stayed for one day...and the rooms were spacious....with minimum staff also they managed it very well...front desk was very co operative...no hassles...only draw back was no vallet parking....but could ignore it.. breakfast spread was awesome....it's living upto it's reputation.....wud definitely recommend...best 5 star property at dirt price... kudosit was a good stay at gokulam and very helpful staff. i am writing this review being at the hotel so that the management takes it seriously. good location and service.impressed with the service of able from ird and chiranjit from myplace.the gym is well equipped and very well maintained.the staff are very supportive.kaviraj,rajkamal,arun,anudev and kiran do a very good job with correct advice and help with the workouts.the best hotel to stay, to have the best food, best varieties.")